___________________________________________________________________________________________________________________________

___________________________________________________________________________________________________________________________

### Insert the correct project ID number as string

In [ ]:
#provide project number ID as string (PO number in string format)
project_id = 'project_test'

___________________________________________________________________________________________________________________________

___________________________________________________________________________________________________________________________

In [ ]:
#import for retrieving predicted gRNAs
from pybiomart import Server
import pandas as pd
import urllib.request
import os
# import nest_asyncio
# nest_asyncio.apply()
from selenium import webdriver
import time
pd.set_option('display.max_rows', None)
pd.options.display.max_columns = None
#imports for primer design
from Bio.Seq import Seq
import primer3
import numpy as np
import requests
import sys
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# define functions
def predict_gRNA(df_genes):
    
    #uses globally declared dataframe, declared prior to function calls
    global gRNA_predicted
    
    for entry in range(len(df_genes)):
        gene_ens_id = df_genes["GENE ID"].iloc[entry]

        #parse dataset for gene_id
        data_gene = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name'],
                      filters={'link_ensembl_gene_id': gene_ens_id})

        #if gene_id found and unique, construct Synthego url
        if len(data_gene.index) == 1:
            #print(data_gene)
            gene_id = data_gene.iat[0,0]
            gene_symbol = data_gene.iat[0,1]
            url = 'https://design.synthego.com/#/design/results?genome=homo_sapiens_gencode_26_primary&nuclease=cas9&gene_id='+gene_id+'&symbol='+gene_symbol
            # specify the url
            urlpage = url
            #print(urlpage)
            #suppress opening Firefox browser window
            os.environ['MOZ_HEADLESS'] = '1'
            # run firefox webdriver from executable path of your choice
            driver = webdriver.Firefox(executable_path = 'D:\Alex\geckodriver-v0.31.0-win64\geckodriver.exe')
            # get web page
            driver.get(urlpage)
            #30 seconds wait to have page fully loaded - might need to increase time
            time.sleep(30)
            results = driver.find_elements("xpath", "//*[@class='ng-binding']")
            #print('Number of results', len(results))
            #Store predictions
            gRNAs_synth = []
            # loop over results
            for result in results:
                #print(result.text)
                gRNAs_synth.append(result.text)
            #extract minimal information for predicted gRNAs
            gRNAs_synth_min = gRNAs_synth[3:9]
            gRNAs_synth_min_df = pd.DataFrame(gRNAs_synth_min[2:], index =[gene_symbol+'-gRNA1', gene_symbol+'-gRNA2', gene_symbol+'-gRNA3', gene_symbol+'-gRNA4'],
                                                 columns =['gRNA sequence'])
            #add column(s) with ENSG number/gene name to df
            gRNAs_synth_min_df['GENE ID'] = gene_id
            gRNAs_synth_min_df['GENE NAME'] = gene_symbol
            #print(gRNAs_synth_min_df)
            gRNA_predicted = pd.concat([gRNA_predicted, gRNAs_synth_min_df])

        else:
            raise SystemExit("Stop right there - ambiguous gene name!")   

In [ ]:
#connect to Biomart Server
server = Server(host='http://www.ensembl.org')

# use #dataset.list_filters() to see available filters for list_filters method
#dataset.list_filters()

#generate dataset (homo sapiens)
dataset = (server.marts['ENSEMBL_MART_ENSEMBL']
                 .datasets['hsapiens_gene_ensembl'])

In [ ]:
#read gene ID numbers from csv file - gene ID in form of ENSG00000141510
df_genes_id = pd.read_csv(project_id + '.csv')
df_genes_id.columns = df_genes_id.columns.str.upper()
df_genes_id.head(10)

In [ ]:
#loop through gene IDs, retrieve gene name (to double check with client data) 
#predict and scrape gRNAs
# create df for storage of predicted gRNAs
gRNA_predicted = pd.DataFrame(columns=['gRNA sequence','Gene ID', 'Gene Name'])

In [ ]:
#call function to retrieve predicted gRNAs
predict_gRNA(df_genes_id)

In [ ]:
#check if all genes have predicted gRNAs - if not, run predicted_gRNA function until all genes have predicted gRNAs)
while len(gRNA_predicted[gRNA_predicted['gRNA sequence'].isnull()]) > 0:
    print(gRNA_predicted[gRNA_predicted['gRNA sequence'].isnull()])
    nan_rows = gRNA_predicted[gRNA_predicted['gRNA sequence'].isnull()]
    nan_rows = nan_rows.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index='False')
    nan_rows = nan_rows[['GENE ID', 'GENE NAME']]
    nan_rows.columns = nan_rows.columns.str.upper()
    predict_gRNA(nan_rows)
    
print(gRNA_predicted[gRNA_predicted['gRNA sequence'].isnull()])

In [ ]:
#clean gRNA_predicted df
gRNA_predicted = gRNA_predicted.drop(['Gene ID', 'Gene Name'], axis=1)
#save predicted gRNAs as csv
gRNA_predicted.to_csv(project_id + '_gRNA_predicted.csv') 
#sanity check df
gRNA_predicted.head(10)

In [ ]:
#replace U with T in gRNA sequences
gRNA_predicted["gRNA_sequence_T"] = gRNA_predicted["gRNA sequence"].str.replace("U",'T')
#add column to indicate forward/reverse orientation of gRNA in relation to gene sequence
gRNA_predicted["gRNA_sequence_T_forward_rel_to_gene"] = gRNA_predicted["gRNA_sequence_T"]

#sanity check the dataframe
gRNA_predicted.head(10)

In [ ]:
#REST API python3 Ensembl
server = "https://rest.ensembl.org"
#initiate dict to store gene sequences for all genes in gRNA_predicted
gene_sequences = {}

# get unique gene IDs from gRNA_predicted
unique_gene_IDs = gRNA_predicted['GENE ID'].unique()

for entry in unique_gene_IDs:
    ext = "/sequence/id/" + entry +"?"
    # retrieve plain text gene sequence
    r = requests.get(server+ext, headers={ "Content-Type" : "text/plain"})
    # sanity check object r
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    gene_sequences[entry] = r.text 
#pd.DataFrame.from_dict(gene_sequences, orient='index')

In [ ]:
#sanity check for orientation and existence of gRNA_sequence_T in gene sequences 
for_rev_orientation_gRNA = []

#using .find to match gRNA sequence_T to gene sequence - if not found output will be -1.
for row in range(len(gRNA_predicted)):
        #get key from gRNA_predicted to retrieve value from gene_sequences dict
        key = gRNA_predicted["GENE ID"].iloc[row]
        value = gene_sequences[key]
        orientation = value.find(gRNA_predicted["gRNA_sequence_T"].iloc[row])
        for_rev_orientation_gRNA.append(orientation)
print(for_rev_orientation_gRNA)

In [ ]:
gRNA_predicted['gRNA reverse (-1 = True)'] = for_rev_orientation_gRNA
#gRNA_predicted

In [ ]:
#In case of output -1 in for_rev_orientation_gRNA use reverse complement gRNA to search for match
for entry in range(len(gRNA_predicted)):
    if gRNA_predicted.iat[entry, 5] == -1:
        gRNA_predicted["gRNA_sequence_T_forward_rel_to_gene"].iloc[entry] = Seq(gRNA_predicted["gRNA_sequence_T"].iloc[entry]).reverse_complement()
        gRNA_predicted["gRNA_sequence_T_forward_rel_to_gene"].iloc[entry] = str(gRNA_predicted["gRNA_sequence_T_forward_rel_to_gene"].iloc[entry])


In [ ]:
#sanity check for orientation and existence of gRNA_sequence_T in gene sequences 
for_orientation_gRNA = []

#using .find to match gRNA sequence_T to gene sequence - if not found output will be -1.
#In case of output -1 use reverse complement gRNA to search for match
for row in range(len(gRNA_predicted)):
        #get key from gRNA_predicted to retrieve value from gene_sequences dict
        key = gRNA_predicted["GENE ID"].iloc[row]
        value = gene_sequences[key]
        orientation = value.find(gRNA_predicted["gRNA_sequence_T_forward_rel_to_gene"].iloc[row])
        for_orientation_gRNA.append(orientation)
print(for_orientation_gRNA)

In [ ]:
gRNA_predicted["Forward position gRNA"] = for_orientation_gRNA

In [ ]:
#Extract gRNA flanking sequences ~250 nt for primer design
#dict to hold sequences
sequences_for_primer_design = []

for row in range(len(gRNA_predicted)):
    beginning = gRNA_predicted['Forward position gRNA'].iloc[row] - 250
    end = gRNA_predicted['Forward position gRNA'].iloc[row] + 250
    key = gRNA_predicted["GENE ID"].iloc[row]
    value = gene_sequences[key]
    seq_slice = value[beginning:end]
    sequences_for_primer_design.append(seq_slice)
#sanity check dict
#sequences_for_primer_design

In [ ]:
gRNA_predicted['Seq for primer design'] = sequences_for_primer_design
gRNA_predicted.head(10)

In [ ]:
#use index to make column named guideRNA name
gRNA_predicted['gRNA name'] = gRNA_predicted.index

In [ ]:
primer_designed = {}
for entry in range(len(gRNA_predicted)):
    seq_dict = {
        'SEQUENCE_ID': gRNA_predicted['gRNA name'].iloc[entry],
        'SEQUENCE_TEMPLATE': gRNA_predicted['Seq for primer design'].iloc[entry],
    }
    primer_designed[gRNA_predicted['gRNA name'].iloc[entry]] = primer3.designPrimers(seq_dict,    
        {
            'PRIMER_OPT_SIZE': 20,
            'PRIMER_PICK_INTERNAL_OLIGO': 1,
            'PRIMER_INTERNAL_MAX_SELF_END': 8,
            'PRIMER_MIN_SIZE': 18,
            'PRIMER_MAX_SIZE': 25,
            'PRIMER_OPT_TM': 60.0,
            'PRIMER_MIN_TM': 57.0,
            'PRIMER_MAX_TM': 63.0,
            'PRIMER_MIN_GC': 20.0,
            'PRIMER_MAX_GC': 80.0,
            'PRIMER_MAX_POLY_X': 100,
            'PRIMER_INTERNAL_MAX_POLY_X': 100,
            'PRIMER_SALT_MONOVALENT': 50.0,
            'PRIMER_DNA_CONC': 50.0,
            'PRIMER_MAX_NS_ACCEPTED': 0,
            'PRIMER_MAX_SELF_ANY': 12,
            'PRIMER_MAX_SELF_END': 8,
            'PRIMER_PAIR_MAX_COMPL_ANY': 12,
            'PRIMER_PAIR_MAX_COMPL_END': 8,
            'PRIMER_PRODUCT_SIZE_RANGE': [[375, 500]],
        })

In [ ]:
primer_df = pd.DataFrame.from_dict(primer_designed, orient='index')
primer_df = primer_df[["PRIMER_LEFT_0_SEQUENCE", "PRIMER_RIGHT_0_SEQUENCE", "PRIMER_PAIR_0_PRODUCT_SIZE", "PRIMER_LEFT_0", "PRIMER_RIGHT_0", "PRIMER_INTERNAL_0", "PRIMER_LEFT_0_TM", "PRIMER_RIGHT_0_TM"]]
primer_df.head(10)

In [ ]:
gRNA_primer_df = gRNA_predicted.join(primer_df)
gRNA_primer_df = gRNA_primer_df.drop(columns=["gRNA name", "gRNA reverse (-1 = True)"])
gRNA_primer_df_wo_seq = gRNA_primer_df.drop(columns=["Seq for primer design"])
gRNA_primer_df_wo_seq.head(10)

In [ ]:
gRNA_primer_df_wo_seq.to_csv(project_id + '_gRNA_primer.csv')